<a href="https://colab.research.google.com/github/Tonelli2003/Aula_2_rpa_ia/blob/main/Aula_2_rpa_ia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import os
import zipfile
import requests
import matplotlib.pyplot as plt

In [ ]:
EXTRAÇÃO ZIP

In [16]:
print("PASSO 1: Verificando seu ZIP...")
zip_path = '/content/dfp_cia_aberta_2025.zip'
if os.path.exists(zip_path):
  print(f"ZIP encontrado: {zip_path}")
else:
  print("ZIP não achado! Upload dfp_cia_aberta_2025.zip")

PASSO 1: Verificando seu ZIP...
ZIP encontrado: /content/dfp_cia_aberta_2025.zip


In [17]:
print("\nPASSO 2: Criando pasta dfp_csvs/...")
extract_to = '/content/dfp_csvs/'
os.makedirs(extract_to, exist_ok=True)
print("Pasta OK!")



PASSO 2: Criando pasta dfp_csvs/...
Pasta OK!


In [19]:
print("\nPASSO 3: Abrindo ZIP...")
with zipfile.ZipFile(zip_path, 'r') as z:
  print(f"ZIP tem {len(z.namelist())} arquivos")
  z.extractall(extract_to)
  print("Extração concluída!")


PASSO 3: Abrindo ZIP...
ZIP tem 19 arquivos
Extração concluída!


In [20]:
print("\nPASSO 4: Listando CSVs importantes...")
csvs = [f for f in os.listdir(extract_to) if f.endswith('.csv')]
demos = [f for f in csvs if any(x in f for x in
['DRE','DMPL','2025.csv'])]
for csv in demos:
  size = os.path.getsize(extract_to+csv)/1e6
  print(f"{csv} ({size:.0f}MB)")


PASSO 4: Listando CSVs importantes...
dfp_cia_aberta_DFC_MI_con_2025.csv (1MB)
dfp_cia_aberta_DFC_MI_ind_2025.csv (1MB)
dfp_cia_aberta_DRE_ind_2025.csv (1MB)
dfp_cia_aberta_DVA_ind_2025.csv (1MB)
dfp_cia_aberta_BPA_ind_2025.csv (2MB)
dfp_cia_aberta_BPP_con_2025.csv (2MB)
dfp_cia_aberta_DFC_MD_ind_2025.csv (0MB)
dfp_cia_aberta_BPP_ind_2025.csv (3MB)
dfp_cia_aberta_composicao_capital_2025.csv (0MB)
dfp_cia_aberta_DRE_con_2025.csv (1MB)
dfp_cia_aberta_DRA_con_2025.csv (0MB)
dfp_cia_aberta_parecer_2025.csv (2MB)
dfp_cia_aberta_DVA_con_2025.csv (1MB)
dfp_cia_aberta_2025.csv (0MB)
dfp_cia_aberta_DMPL_con_2025.csv (6MB)
dfp_cia_aberta_BPA_con_2025.csv (1MB)
dfp_cia_aberta_DMPL_ind_2025.csv (6MB)
dfp_cia_aberta_DFC_MD_con_2025.csv (0MB)
dfp_cia_aberta_DRA_ind_2025.csv (0MB)


CARREGAR DRE + LUCRO

In [38]:
# Seu arquivo DRE
dre_path = '/content/dfp_csvs/dfp_cia_aberta_DRE_con_2025.csv'
print("PASSO 1: Carregando DRE CVM...")
df_dre = pd.read_csv(dre_path, sep=';', encoding='latin-1')
print(f"{len(df_dre):,} linhas de {df_dre['DENOM_CIA'].nunique()} empresas")

PASSO 1: Carregando DRE CVM...
3,260 linhas de 43 empresas


In [39]:
# PASSO 2: Filtrar só Lucro Líquido!
print("\n PASSO 2: Procurando '3.11' = Lucro")
lucro_df = df_dre[df_dre['CD_CONTA'] == '3.11']
print(f" {len(lucro_df)} linhas lucro encontradas!")


 PASSO 2: Procurando '3.11' = Lucro
 82 linhas lucro encontradas!


In [41]:
# PASSO 3: Último trimestre por CNPJ
print("\nPASSO 3: Mais recente por empresa")
lucro_latest = lucro_df.sort_values('DT_REFER').groupby('CNPJ_CIA').last()
print(f" {len(lucro_latest)} empresas Q3'25!")


PASSO 3: Mais recente por empresa
 41 empresas Q3'25!


In [49]:
#PASSO 4: Tabela TOP 5 linda
top5 = lucro_latest.nlargest(5,'VL_CONTA')[['DENOM_CIA','VL_CONTA','DT_REFER']]
top5['Lucro R$'] = (top5['VL_CONTA']*1_000).map('R$ {:,.0f}'.format)
print("\nTOP 5 LUCRO Q3'25:")
print(top5[['DENOM_CIA','Lucro R$','DT_REFER']].to_markdown(index=False))


TOP 5 LUCRO Q3'25:
| DENOM_CIA                       | Lucro R$          | DT_REFER   |
|:--------------------------------|:------------------|:-----------|
| PETTENATI S.A. INDUSTRIA TEXTIL | R$ 57,768,709,000 | 2025-06-30 |
| BCO BRADESCO S.A.               | R$ 23,924,636,000 | 2025-12-31 |
| BCO BRASIL S.A.                 | R$ 16,781,938,000 | 2025-12-31 |
| AMBEV S.A.                      | R$ 15,988,433,000 | 2025-12-31 |
| SUZANO S.A.                     | R$ 13,437,687,000 | 2025-12-31 |


CARREGAR DMPL - Patrimônio Líquido

In [50]:
print("PASSO 1: Carregando DMPL CVM...")
dmpl_path = '/content/dfp_csvs/dfp_cia_aberta_DMPL_con_2025.csv'
df_dmpl = pd.read_csv(dmpl_path, sep=';', encoding='latin-1',
low_memory=False)

PASSO 1: Carregando DMPL CVM...


In [51]:
print(f"DMPL carregada: {len(df_dmpl):,} linhas")
print("Cabeçalhos:", df_dmpl.columns.tolist()[:8], "...")

DMPL carregada: 22,837 linhas
Cabeçalhos: ['CNPJ_CIA', 'DT_REFER', 'VERSAO', 'DENOM_CIA', 'CD_CVM', 'GRUPO_DFP', 'MOEDA', 'ESCALA_MOEDA'] ...


In [52]:
print("\nPASSO 2: Filtrando Patrimônio Líquido (CD_CONTA '5.05')")
pl_df = df_dmpl[df_dmpl['CD_CONTA'] == '5.05'].copy()
pl_latest = pl_df.sort_values('DT_REFER').groupby('CNPJ_CIA').last()


PASSO 2: Filtrando Patrimônio Líquido (CD_CONTA '5.05')


In [53]:
print(f"PL 5.05 encontrado: {len(pl_latest)} empresas")
print("\nTOP 5 Patrimônio Líquido Q3'25:")
print(pl_latest[['DENOM_CIA', 'VL_CONTA',
'DT_REFER']].round(0).head().to_string())


PL 5.05 encontrado: 43 empresas

TOP 5 Patrimônio Líquido Q3'25:
                                                 DENOM_CIA    VL_CONTA    DT_REFER
CNPJ_CIA                                                                          
00.000.000/0001-91                         BCO BRASIL S.A.  17305621.0  2025-12-31
01.027.058/0001-91  CIELO S.A. -  INSTITUIÇÃO DE PAGAMENTO         0.0  2025-12-31
01.083.200/0001-18                         NEOENERGIA S.A.     26000.0  2025-12-31
01.968.293/0001-68                        ELETRORIVER S.A.    -22692.0  2025-12-31
02.149.205/0001-69                       PORTO SEGURO S.A.     -3316.0  2025-12-31


In [54]:
pl_medio = pl_latest['VL_CONTA'] / 1e6 # Em R$ milhões
print(f"\nExemplo: PL médio ~R${pl_medio.mean():,.0f}M")
print("CNPJ_CIA pronto para ligar com lucros DRE!")


Exemplo: PL médio ~R$1M
CNPJ_CIA pronto para ligar com lucros DRE!


ROE

In [55]:
print("1️-Tabela vazia para ROE...")
df_roe = pd.DataFrame(index=lucro_latest.index)
print("OK! Mesmos CNPJ\n")

1️-Tabela vazia para ROE...
OK! Mesmos CNPJ



In [63]:
print("2. Coluna Lucro (Cell2)")
print("Coluna Lucro do lucro_latest...")


df_roe['Lucro R$'] = lucro_latest['VL_CONTA'] / 1e6


cnpj_bb = '00.000.000/0001-91'

try:
    valor_bb = df_roe.loc[cnpj_bb, 'Lucro R$']
    print(f"BB: {valor_bb} bi\n")
except KeyError:
    print(f"Aviso: CNPJ {cnpj_bb} não encontrado no índice.")

2. Coluna Lucro (Cell2)
Coluna Lucro do lucro_latest...
BB: 16.781938 bi



In [68]:
print("Coluna PL do pl_medio...")

df_roe['PL R$'] = pl_medio.reindex(df_roe.index).fillna(100)

cnpj_exemplo = '00.000.000/0001-91'

try:
    valor_pl = df_roe.loc[cnpj_exemplo, 'PL R$']
    print(f"BB: {valor_pl} unidades de medida\n")
except KeyError:
    print(f"CNPJ {cnpj_exemplo} não encontrado no DataFrame df_roe.\n")

Coluna PL do pl_medio...
BB: 17.305621 unidades de medida



In [69]:

print("FÓRMULA: (Lucro / PL) * 100")

df_roe['ROE_%'] = (df_roe['Lucro R$'] / df_roe['PL R$'] * 100).round(1)

cnpj_bb = '00.000.000/0001-91'

try:
    resultado_roe = df_roe.loc[cnpj_bb, 'ROE_%']
    print(f"BB ROE: {resultado_roe}% \n")
except KeyError:
    print(f"Erro: O CNPJ {cnpj_bb} não foi encontrado no índice do DataFrame.\n")

FÓRMULA: (Lucro / PL) * 100
BB ROE: 97.0% 



In [71]:
total_ruim = len(lucro_latest) - len(df_roe[df_roe['PL R$'] > 0.1])

top10 = df_roe.sort_values('ROE_%', ascending=False).head(10).reset_index()

relatorio_audit = pd.DataFrame({
    'Total_Empresas': len(lucro_latest),
    'PL_Ruim': total_ruim,
    '%_OK': f"{(len(df_roe) - total_ruim) / len(lucro_latest) * 100:.1f}%"
}, index=[0])

top10.to_excel('ROE_Limpo.xlsx', index=False)
relatorio_audit.to_excel('Audit_Dados_CVM.xlsx', index=False)

from google.colab import files
try:
    files.download('ROE_Limpo.xlsx')
    files.download('Audit_Dados_CVM.xlsx')
    print("✅ Sucesso: 2 Excels gerados (ROE + AUDIT)!")
except Exception as e:
    print(f"Arquivos salvos localmente, mas o download automático falhou: {e}")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ Sucesso: 2 Excels gerados (ROE + AUDIT)!
